# Aufgabe 3: Data Preparation und Modeling 

## TODO:
- Anomalien bereinigen
- Feature Engineering
    - Unnötige Spalten (siehe Aufg. 2) entfernen
    - One-Hot Encoding einiger Eigenschaften
- 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import math
import seaborn as sb
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [ ]:
dataset = pd.read_csv('./DatenAusgegeben1.0.csv', delimiter=';', encoding = 'cp852')
dataset

## Unnötige Spalten entfernen
Zu entfernen sind:  
Siehe Aufg. 2. Untersuchung auf einen Zusammenhang zwischen Preis und Verkaufszeitpunkt:
- Verkaufsjahr
- Verkaufsmonat

(Da Gesamtwohnfläche zusammengesetzt aus erster Stock und zweiter Stock ist:
- Erster Stock in qm
- Zweiter Stock in qm
#TODO: Prüfen ob nicht rauszunehmen)

Siehe Aufg. 2. Untersuchung des Datensatzes auf vernachlässigbare Hauseigenschaften (Spalten):
- Pool
- Küchen
- Klimaanlage
- Heizung

In [ ]:
df_keinunnoetig = dataset.drop(['Verkaufsjahr', 'Verkaufsmonat', 'Erster Stock in qm', 'Zweiter Stock in qm', 'Pool', 'Küchen', 'Klimaanlage', 'Heizung'], axis = 1)

## Anomalien bereinigen
Nun sollen Anomalien in dem Datensatz gefunden und bereinigt werden.
Hierbei wird der Interquartilsabstand verwendet um Ausreißer zu finden.  
(https://en.wikipedia.org/wiki/Interquartile_range)  

Hierbei sollen ausschließlich numerische Werte, also keine Werte einer ordinalen Skala betrachtet werden. (`columns_filtered`)

In [ ]:
columns = ['Grundstück in qm', 'Zustand', 'Gebaut',
           'Renoviert', 'Zustand Fassade', 'Kellerflche in qm',
           'Wohnflche in qm', 'Schlafzimmer', 'Küchenqualitt', 
           'Rume', 'Garagenkapazitt', 'Preis']

columns_filtered = ['Grundstück in qm', 'Kellerflche in qm', 'Wohnflche in qm', 
           'Schlafzimmer', 'Gebaut', 'Rume', 'Garagenkapazitt', 'Preis']

Q1 = df_keinunnoetig[columns_filtered].quantile(0.25)
Q3 = df_keinunnoetig[columns_filtered].quantile(0.75)
IQR = Q3 - Q1

df_anomalienbereinigen = df_keinunnoetig[~((df_keinunnoetig[columns_filtered] < (Q1 - 1.5 * IQR)) |(dataset[columns_filtered] > (Q3 + 1.5 * IQR))).any(axis=1)]

## Verifikation der Bereinigung
Nun soll verifiziert werden, ob die Bereinigung auf Anomalien erfolgreich gewesen ist. Hierzu können die untenstehenden Boxplots mit denen aus **Aufg. 2, Finden von Anomalien** verglichen werden.

In [ ]:
fig, axes = plt.subplots(4, 5, figsize = (10, 15))
i = 0
plt.tight_layout(pad = 2)
for column in columns:
  chosenax = axes[math.floor(i / 5)][i % 5]
  chosenax.set_title(column)
  sb.boxplot(data = df_anomalienbereinigen[[column]], ax = chosenax)
  i = i + 1
plt.show()

In [ ]:
encode_kategorien = ['Bezirk', 'Lage', 'Heizungsqualitt', 'Typ', 'Steigung', 'Grundstücksform', 'Garage Typ', 'Zone']

encoded_kategorieeigenschaften = pd.get_dummies(dataset[encode_kategorien].fillna('Keine'))
df_encoded = df_anomalienbereinigen.drop(encode_kategorien, axis = 1).join(encoded_kategorieeigenschaften)

In [ ]:
reg = LinearRegression()
df_train = df_encoded.drop('Preis', axis = 1)
values_predict = df_encoded['Preis']
x_train, x_test, y_train, y_test = train_test_split(df_train, values_predict, test_size = 0.2)

reg.fit(x_train, y_train)

print(reg.coef_, reg.intercept_, reg.rank_)
reg.score(x_test, y_test)